# Markov Decision Process using Value Iteration

In this notebook, a simple roll dice game will be modeled as a MDP problem.  The framework of mdptoolbox will be used to 
solve the game and Value Iteration used to find the optimal policy and expected value of the initial state given.

### Model

#### States
The states will represent the earnings of the agent.  Thus, the agent starts at state 0 representing 0 initial earnings.
The agent can then transition to other states or earnings by rolling the die and gaining rewards.
#### Actions
The actions represent categories of transitions between states. The transition matrix will have two actions one rolling
the die and one for leaving the game.

### Simple Coin Toss Example

In a simple coin toss game, you can choose to flip the coin or leave the game and take your earnings.  If you decide to
 flip you gain a reward of 1 if heads or if tails you will lose all of your earnings.

Below illustrates the state and transitions for this markov decision process.

![markov example](./markov.png)
### Initialization
We will use the mdptoolbox which is a python package for solving MDP problems as well as numpy which is a library which
 provides abstractions for matrix and array manipulations.

In [15]:
import mdptoolbox
import numpy as np

# the number of sides of the dice/coin
n_sides = 2

# the number of runs to simulate
n_runs = 2

# the number of actions
n_actions = 2

# beginning and ending states
n_initial_terminal_states = 2

# the number of total states: the - n_runs is due to the fact that two of the states
# end up in the terminal state so we can subtract them from the number of overall states
n_states = n_runs * n_sides + n_initial_terminal_states  - n_sides

# the boolean mask to indicate which states you will loose money on
isBadSide = np.array([0]*n_states)

isGoodSide = [not i for i in isBadSide]

# the array which contains the values of the die
die = np.array([1] * n_states)

# the total earnings given a die roll
earnings = die * isGoodSide  # [1, 1]

# Calculate probability for Input:
probability_dice = 1.0 / n_sides

#### Transitions
The transition matrix represents all of the possible transitions between all of the states.

There are two actions, thus the transition matrix will be of size 2.  

For each, action there will be a probability of transitioning between each state.  Thus the transition matrix will be 
of n_actions * n_states * n_states.

In [2]:
transition_matrix = np.zeros([n_actions, n_states, n_states])
print(transition_matrix)

[[[0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]]

 [[0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]]]


### Action: Leave the Game 
Probabilities of the transitions of leaving the game and keeping the same score.  There is 100% chance that you can remain
in the same state.

In [3]:
for i in range(len(transition_matrix[0])):
    transition_matrix[0][i][i] = 1
            
print(transition_matrix[0])

[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]


### Action: Roll the Dice
The probabilities of transitions of rolling the die.  From the first state there is a .5 chance of going to state 1 and 
.5 chance of getting tails and going to the terminal state.  The third and fourth state are terminal states with no chance
of the agent moving from from any other state than the ending state.

In [4]:
for i in range(len(transition_matrix[1]) -2):
    transition_matrix[1][i][i+1] = .5
    transition_matrix[1][i][-1] = .5

for i in range(len(transition_matrix[1])-2, len(transition_matrix[1])):
    transition_matrix[1][i][-1] = 1

print(transition_matrix[1])

[[0.  0.5 0.  0.5]
 [0.  0.  0.5 0.5]
 [0.  0.  0.  1. ]
 [0.  0.  0.  1. ]]


### Reward: Initialization and Leaving the Game
The reward matrix is the same dimension as the transition matrix.
There is no reward for leaving the game.  Rewards are only accumulated by rolling the die.

In [5]:
reward_matrix = np.zeros([n_actions, n_states, n_states])

### Reward: Rolling the Die
Rolling the die can result in gaining a reward if it was heads or losing everything.  The ending column describes
the result of rolling a tails and losing the accumulated reward.

In [6]:
reward_acc = 1
for i in range(len(transition_matrix[1])-1):
    reward_matrix[1][i][i+1] = reward_acc
    reward_matrix[1][i][-1] = (1- reward_acc)
    reward_acc+=1
    
reward_matrix[1][len(transition_matrix[1])-1][len(transition_matrix[1])-1] = (1 - reward_acc)

print(reward_matrix[1])

[[ 0.  1.  0.  0.]
 [ 0.  0.  2. -1.]
 [ 0.  0.  0. -2.]
 [ 0.  0.  0. -3.]]


### Initializing the MDP: Value Iteration Model
Using the mdptoolbox the Value Iteration Model is initiated and run with the given transition and reward matricies.

In [7]:
discount_factor = .99999  # less than 1 for convergence
vi = mdptoolbox.mdp.ValueIteration(transition_matrix, reward_matrix, discount_factor)
vi.run()

The optimal policy indicates which action to take in each state.  The expected_values indicates what is the
value of the state and how many we points we can expect by following the optimal policy.
                
The optimal policy and expected value are as follows:

In [8]:
optimal_policy = vi.policy
expected_values = vi.V

print optimal_policy
print expected_values

(1, 1, 0, 0)
(0.7499975, 0.5, 0.0, 0.0)


From the optimal policy:  (1, 1, 0, 0) we can see that we should choose action 1 or flip the coin in the initial state
and the second state or the first two flips.

The value of each state is: (0.7499975, 0.5, 0.0, 0.0).  The value at the initial state is .75, .5 in the second state.

## N-Die Roll Game
The same game is played with the sides of the die at N=6.  The isBadSide = [1, 1, 1, 0 , 0, 0] or in other words
the only rolls which will be rewarded is the complement or when a 4, 5, or 6 is rolled.

### Initialization

In [16]:
# the number of sides of the dice
n_sides = 6

# the number of runs to simulate
n_runs = 2

# the number of actions
n_actions = 2

# beginning and ending states
n_initial_terminal_states = 2

# the number of total states
n_states = n_runs * n_sides + n_initial_terminal_states  # from 0 to 2N, plus quit

# the boolean mask to indicate which states you will loose money on
isBadSide = np.array([1, 1, 1, 0, 0, 0])
isGoodSide = [not i for i in isBadSide]

# the array which contains the values of the die
die = np.arange(1, n_sides + 1)  # [1, 2, 3, 4, 5, 6]

# the total earnings given a die roll
earnings = die * isGoodSide  # [0, 0, 0, 4, 5, 6]

# Calculate probability for Input:
probability_dice = 1.0 / n_sides

### Action: Do not Roll the Dice

There 100% chance that you do not have to roll the dice if you are in a given state and you will not transition to
another state but will remain in the same one.

In [10]:
transition_matrix = np.zeros([n_actions, n_states, n_states])
# the probability matrix for the first action, if you do not roll
for i in range(len(transition_matrix[0])):
    transition_matrix[0][i][i] = 1
            
print(transition_matrix[0])

[[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]]


### Action: Roll the Dice
The first row gives the probabilities of transitioning from the initial state to the possible states of valid dice
rolls and the state of rolling a game ending dice roll which is represented by the last column.  In this example, there 
is a p chance of rolling a 4, 5, 6, and a 1/2 chance of transitioning to the final state which is losing,
all of the money earned thus far (which is 0 at state 0).

The second row gives all of the possible transitions from the second state.  We can see that the the same ratio exists
from transitioning, but is shifted down since the minimum valid state is given by rolling two 4's.

In [11]:
#if roll
p=1.0/n_sides
min_roll = 4
# after the first roll, you have a 1/6 chance of transistioning 
for i in range(len(transition_matrix[1]) -1):
    for j in range(1, min(4, len(transition_matrix[1])- i)):
        transition_matrix[1][i][i+j] = p
        transition_matrix[1][i][-1] = 1 - sum(transition_matrix[1][i][:-1])

for i in range(len(transition_matrix[1])-2, len(transition_matrix[1])):
    transition_matrix[1][i][-1] = 1

print(transition_matrix[1])

np.sum(transition_matrix[0],axis=1)
np.sum(transition_matrix[1],axis=1)

[[0.         0.16666667 0.16666667 0.16666667 0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.5       ]
 [0.         0.         0.16666667 0.16666667 0.16666667 0.
  0.         0.         0.         0.         0.         0.
  0.         0.5       ]
 [0.         0.         0.         0.16666667 0.16666667 0.16666667
  0.         0.         0.         0.         0.         0.
  0.         0.5       ]
 [0.         0.         0.         0.         0.16666667 0.16666667
  0.16666667 0.         0.         0.         0.         0.
  0.         0.5       ]
 [0.         0.         0.         0.         0.         0.16666667
  0.16666667 0.16666667 0.         0.         0.         0.
  0.         0.5       ]
 [0.         0.         0.         0.         0.         0.
  0.16666667 0.16666667 0.16666667 0.         0.         0.
  0.         0.5       ]
 [0.         0.         0.         0.         0.         0.
  0.         0.16666667 0.16666667 0.16666667 

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

### Reward: Initialization and Leaving the Game
The reward matrix is the same dimension as the transition matrix.
There is no reward for leaving the game.  Rewards are only accumulated by rolling the die.

In [12]:
reward_matrix = np.zeros([n_actions, n_states, n_states])

tot_reward = 0

for i in range(n_states):
    reward_matrix[0][i][i] = tot_reward
    if i == 0:
        tot_reward+=4
    else:
        tot_reward+=1
print(reward_matrix)
            

[[[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
  [ 0.  4.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
  [ 0.  0.  5.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
  [ 0.  0.  0.  6.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
  [ 0.  0.  0.  0.  7.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
  [ 0.  0.  0.  0.  0.  8.  0.  0.  0.  0.  0.  0.  0.  0.]
  [ 0.  0.  0.  0.  0.  0.  9.  0.  0.  0.  0.  0.  0.  0.]
  [ 0.  0.  0.  0.  0.  0.  0. 10.  0.  0.  0.  0.  0.  0.]
  [ 0.  0.  0.  0.  0.  0.  0.  0. 11.  0.  0.  0.  0.  0.]
  [ 0.  0.  0.  0.  0.  0.  0.  0.  0. 12.  0.  0.  0.  0.]
  [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0. 13.  0.  0.  0.]
  [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. 14.  0.  0.]
  [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. 15.  0.]
  [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. 16.]]

 [[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
  [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
  [ 0.  0.  0.  0.  0.  0.  0.  0.  0.

### Reward: Rolling the Die
Rolling the die can result in gaining a reward if it was heads or losing everything.  The ending column describes
the result of rolling a tails and losing the accumulated reward.

In [13]:
#if roll
reward_acc = 4
reward_curr = 4
reward_tot = 0

for i in range(n_states-1):
    reward_curr = reward_acc
    for j in range(1, min(min_roll, n_states - i)):
        reward_matrix[1][i][i + j] = reward_curr
        reward_curr +=1
    
    reward_matrix[1][i][-1] = -reward_tot
    if i ==0:
        reward_tot+=4
    else:
        reward_tot+=1

reward_matrix[1][n_states -1][n_states -1] = -reward_tot 

print(reward_matrix[1])

[[  0.   4.   5.   6.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   4.   5.   6.   0.   0.   0.   0.   0.   0.   0.   0.  -4.]
 [  0.   0.   0.   4.   5.   6.   0.   0.   0.   0.   0.   0.   0.  -5.]
 [  0.   0.   0.   0.   4.   5.   6.   0.   0.   0.   0.   0.   0.  -6.]
 [  0.   0.   0.   0.   0.   4.   5.   6.   0.   0.   0.   0.   0.  -7.]
 [  0.   0.   0.   0.   0.   0.   4.   5.   6.   0.   0.   0.   0.  -8.]
 [  0.   0.   0.   0.   0.   0.   0.   4.   5.   6.   0.   0.   0.  -9.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   4.   5.   6.   0.   0. -10.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.   4.   5.   6.   0. -11.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   4.   5.   6. -12.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   4.   5. -13.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   4. -14.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0. -15.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.

### Initializing the MDP: Value Iteration Model
Using the mdptoolbox the Value Iteration Model is initiated and run with the given transition and reward matricies.

In [14]:
vi = mdptoolbox.mdp.ValueIteration(transition_matrix, reward_matrix, discount_factor)
vi.run()

optimal_policy = vi.policy
expected_values = vi.V

print optimal_policy
print expected_values

(1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0)
(641.6541064639006, 639.2682060821224, 638.4270721487384, 637.6415829511734, 636.9527806407235, 636.3802480802706, 635.92863063626, 635.8199554497721, 635.945048365556, 636.2189288751787, 638.1679278105682, 639.6956433662098, 640.8622583662097, 671.8622583662097)


### Results
Running the simulation, the optimal policy is to roll the die initially and if you have 